# Emotion Classification in Video

## 1. Google Drive Mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import zipfile

path = '/content/drive/MyDrive/colab/Computer-Vision-Course/Data/Datasets/fer_images.zip'
zip_object = zipfile.ZipFile(file=path, mode='r')
zip_object.extractall('./')
zip_object.close()

## 2. Create Train and Test Datasets

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rotation_range=10,  # Degree range for random rotations
                                     zoom_range=0.2,  # Float or [lower, upper]. Range for random zoom. If a float, [lower, upper] = [1-zoom_range, 1+zoom_range]
                                     horizontal_flip=True,  # Randomly flip inputs horizontally
                                     rescale=1/255)  # Rescaling by 1/255 to normalize

train_dataset = train_generator.flow_from_directory(directory='/content/fer2013/train',
                                                    target_size=(48, 48),  # Tuple of integers (height, width), defaults to (256, 256)
                                                    class_mode='categorical',
                                                    batch_size=16,  # Size of the batches of data (default: 32)
                                                    shuffle=True,  # Whether to shuffle the data (default: True) If set to False, sorts the data in alphanumeric order
                                                    seed=10) 

Found 28709 images belonging to 7 classes.


In [4]:
test_generator = ImageDataGenerator(rescale=1/255)

test_dataset = test_generator.flow_from_directory(directory='/content/fer2013/validation',
                                                  target_size=(48, 48),
                                                  class_mode='categorical',
                                                  batch_size=1,
                                                  shuffle=False,
                                                  seed=10)

Found 3589 images belonging to 7 classes.


## 3. Build Convolutional Neural Network(CNN)

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization

num_classes = 7
num_detectors = 32
width, height = 48, 48

network = Sequential()

network.add(Conv2D(filters=num_detectors, kernel_size=3, activation='relu', padding='same', input_shape=(width, height, 3)))
network.add(BatchNormalization())
network.add(Conv2D(filters=num_detectors, kernel_size=3, activation='relu', padding='same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2, 2)))
network.add(Dropout(0.2))

network.add(Conv2D(2*num_detectors, 3, activation='relu', padding='same'))
network.add(BatchNormalization())
network.add(Conv2D(2*num_detectors, 3, activation='relu', padding='same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2, 2)))
network.add(Dropout(0.2))

network.add(Conv2D(2*2*num_detectors, 3, activation='relu', padding='same'))
network.add(BatchNormalization())
network.add(Conv2D(2*2*num_detectors, 3, activation='relu', padding='same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2, 2)))
network.add(Dropout(0.2))

network.add(Conv2D(2*2*2*num_detectors, 3, activation='relu', padding='same'))
network.add(BatchNormalization())
network.add(Conv2D(2*2*2*num_detectors, 3, activation='relu', padding='same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2, 2)))
network.add(Dropout(0.2))

network.add(Flatten())

network.add(Dense(2*2*num_detectors, activation='relu'))
network.add(BatchNormalization())
network.add(Dropout(0.2))

network.add(Dense(2*num_detectors, activation='relu'))
network.add(BatchNormalization())
network.add(Dropout(0.2))

network.add(Dense(num_classes, activation='softmax'))

## 4. Train the Model

In [6]:
network.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 70
network.fit(train_dataset, epochs=epochs)

Epoch 1/70
1795/1795 [==============================] - 73s 34ms/step - loss: 1.8591 - accuracy: 0.2677
Epoch 2/70
1795/1795 [==============================] - 58s 32ms/step - loss: 1.5498 - accuracy: 0.3927
Epoch 3/70
1795/1795 [==============================] - 58s 32ms/step - loss: 1.3944 - accuracy: 0.4654
Epoch 4/70
1795/1795 [==============================] - 57s 32ms/step - loss: 1.3154 - accuracy: 0.5013
Epoch 5/70
1795/1795 [==============================] - 60s 34ms/step - loss: 1.2588 - accuracy: 0.5276
Epoch 6/70
1795/1795 [==============================] - 58s 32ms/step - loss: 1.2175 - accuracy: 0.5432
Epoch 7/70
1795/1795 [==============================] - 59s 33ms/step - loss: 1.1829 - accuracy: 0.5591
Epoch 8/70
1795/1795 [==============================] - 61s 34ms/step - loss: 1.1589 - accuracy: 0.5676
Epoch 9/70
1795/1795 [==============================] - 58s 32ms/step - loss: 1.1384 - accuracy: 0.5780
Epoch 10/70
1795/1795 [==============================] - 57s 32m

## 5. Classify Emotions in Video

In [7]:
import cv2

# Create VideoCapture object
cap = cv2.VideoCapture('/content/drive/MyDrive/colab/Computer-Vision-Course/Data/Videos/emotion_classification.mp4')

video_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)  # Video capture's frame width
video_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # Video capture's frame height
video_size = (round(video_width), round(video_height)) # Video size
video_fps = cap.get(cv2.CAP_PROP_FPS)  # FPS(Frames Per Second)
frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Number of frames

print('Number of frames:', frame_cnt, '/ FPS:', round(video_fps), '/ Frame size:', video_size)

Number of frames: 408 / FPS: 24 / Frame size: (640, 360)


- `connected` means whether it has been read normally.
- `frame` is the first frame of video. 

In [8]:
# In Linux, the extension of video output must be set to avi
video_output_path = '/content/drive/MyDrive/colab/Computer-Vision-Course/Data/Videos/emotion_classification_result.avi'

codec = cv2.VideoWriter_fourcc(*'XVID')  # Set the codec  

video_writer = cv2.VideoWriter(video_output_path, codec, video_fps, video_size)

In [9]:
import dlib

cnn_face_detector = dlib.cnn_face_detection_model_v1('/content/drive/MyDrive/colab/Computer-Vision-Course/Data/Weights/mmod_human_face_detector.dat')

green_color=(0, 255, 0)
red_color=(0, 0, 255)
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

while (cv2.waitKey(1) < 0):
    connected, frame = cap.read()  # Read one frame from a VideoCapture object
    if not connected:
        break
    face_detections = cnn_face_detector(frame, 1)
    if len(face_detections) > 0:
        for face_detection in face_detections:
            left, top, right, bottom, confidence = face_detection.rect.left(), face_detection.rect.top(), face_detection.rect.right(), face_detection.rect.bottom(), face_detection.confidence
            cv2.rectangle(frame, (left, top), (right, bottom), green_color, 2)
            roi = frame[top:bottom, left:right]
            roi = cv2.resize(roi, (48, 48))  # Extract region of interest from image
            roi = roi / 255  # Normalize
            roi = np.expand_dims(roi, axis=0)
            preds = network.predict(roi)

            if preds is not None:
                pred_emotion_index = np.argmax(preds)
                cv2.putText(frame, emotions[pred_emotion_index], (left, top-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, red_color, 1)
    
    video_writer.write(frame)

video_writer.release()
cap.release()

1/1 [==============================] - 0s 22ms/step
